In [3]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import pandas as pd
import swifter

pd.set_option("display.max_columns", None)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

plt.rcParams["lines.linewidth"] = 2
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Helvetica"
plt.rcParams["text.usetex"] = True
plt.rcParams["font.size"] = 16
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["figure.dpi"] = 150
plt.rcParams["savefig.bbox"] = "tight"
from tqdm.auto import tqdm

import glob
from pathlib import Path

import h3.api.numpy_int as h3
from stc_unicef_cpi.data import process_geotiff as pg


# Reduce full dataset to NGA

In [10]:
df = pd.read_csv(
    "/Users/johnf/Downloads/raw_low_res_dssg/dhs/childpoverty_microdata_gps_21jun22.csv"
)


/var/folders/fc/4tmtzm3n327_sp7y205dwl4h0000gn/T/ipykernel_63577/263308421.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/johnf/Downloads/raw_low_res_dssg/dhs/childpoverty_microdata_gps_21jun22.csv')


In [11]:
nga_df = df[df["countrycode"].str.strip() == "NGA"]


In [14]:
nga_df.dropna(subset=["LATNUM", "LONGNUM"], inplace=True)


/var/folders/fc/4tmtzm3n327_sp7y205dwl4h0000gn/T/ipykernel_63577/1755469988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nga_df.dropna(subset=['LATNUM','LONGNUM'], inplace=True)


In [39]:
nga_df["hex_code"] = nga_df[["LATNUM", "LONGNUM"]].apply(
    lambda row: h3.geo_to_h3(row["LATNUM"], row["LONGNUM"], 7), axis=1
)


In [40]:
nga_df = nga_df.reset_index().drop(columns=["index"]).copy()


In [41]:
nga_df.to_csv(
    "/Users/johnf/Downloads/raw_low_res_dssg/dhs/clean_nga_dhs.csv", index=False
)


# Load preprocessed NGA data

In [24]:
nga_df = pd.read_csv("/Users/johnf/Downloads/raw_low_res_dssg/dhs/clean_nga_dhs.csv")


In [25]:
nga_df.hex_code.value_counts().describe()


count    1358.000000
mean       70.827688
std        34.331554
min         6.000000
25%        45.000000
50%        64.000000
75%        92.000000
max       472.000000
Name: hex_code, dtype: float64

In [26]:
nga_df = nga_df.groupby(by=["hex_code"], as_index=False).mean()


In [27]:
# absolute path to search for all tiff files inside a specified folder
path = r"/Users/johnf/Downloads/raw_low_res_dssg/*.tif"
tif_files = glob.glob(path)
print(tif_files)


['/Users/johnf/Downloads/raw_low_res_dssg/cpiPopData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiElevationData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiSlopeData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiPollutionData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiNDWIData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiNDVIData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiCopLandData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiNighttimeData.tif', '/Users/johnf/Downloads/raw_low_res_dssg/cpiGHSLData.tif']


In [28]:
for i, fname in enumerate(tif_files):
    title = Path(fname).name.lstrip("cpi").rstrip(".tif")
    print(f"Working with {title}: {i+1}/{len(tif_files)}...")
    # Convert to dataframe
    tmp = pg.geotiff_to_df(fname)
    print("Converted to dataframe!")
    print("Dataframe info:")
    print(tmp.info())
    print("Adding hex info...")
    tmp["hex_code"] = tmp[["latitude", "longitude"]].swifter.apply(
        lambda row: h3.geo_to_h3(row["latitude"], row["longitude"], 7), axis=1
    )
    tmp.drop(columns=["latitude","longitude"],inplace=True)
    print("Done!")
    print("Summing within cells...")
    tmp = tmp.groupby(by=["hex_code"], as_index=False).sum()
    print("Joining to survey data...")
    # Aggregate ground truth to hexagonal cells with mean
    # NB automatically excludes missing data for households,
    # so differing quantities of data for different values
    nga_df = nga_df.merge(
        tmp.groupby(by=["hex_code"], as_index=False).sum(), how="left", on="hex_code"
    )
    print("Done!")


Working with PopData: 1/9...
The crs of cpiPopData.tif is: EPSG:4326
The nodatavalue of cpiPopData.tif is: None
The shape of cpiPopData.tif is: (13, 1073, 1338)
The spatial resolution for cpiPopData.tif is: (0.008983152841170794, -0.008983152841219635)
The metadata for cpiPopData.tif is: {'scale_factor': 1.0, 'add_offset': 0.0, 'long_name': ('population', 'M_0', 'M_1', 'M_5', 'M_10', 'M_15', 'M_20', 'F_0', 'F_1', 'F_5', 'F_10', 'F_15', 'F_20')}
Converted to dataframe!
Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927146 entries, 0 to 927145
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   latitude    927146 non-null  float64
 1   longitude   927146 non-null  float64
 2   F_0         927146 non-null  float32
 3   F_1         927146 non-null  float32
 4   F_10        927146 non-null  float32
 5   F_15        927146 non-null  float32
 6   F_20        927146 non-null  float32
 7   F_5         92714

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Done!
Summing within cells...
Joining to survey data...
Done!
Working with ElevationData: 2/9...
The crs of cpiElevationData.tif is: EPSG:4326
The nodatavalue of cpiElevationData.tif is: None
The shape of cpiElevationData.tif is: (1, 1073, 1338)
The spatial resolution for cpiElevationData.tif is: (0.008983152841195215, -0.008983152841195215)
The metadata for cpiElevationData.tif is: {'scale_factor': 1.0, 'add_offset': 0.0, 'long_name': 'elevation'}
1 bands found in ElevationData
Converted to dataframe!
Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435674 entries, 0 to 1435673
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   latitude   1435674 non-null  float64
 1   longitude  1435674 non-null  float64
 2   Elevation  1435674 non-null  float32
dtypes: float32(1), float64(2)
memory usage: 27.4 MB
None
Adding hex info...


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Done!
Summing within cells...
Joining to survey data...
Done!
Working with SlopeData: 3/9...
The crs of cpiSlopeData.tif is: EPSG:4326
The nodatavalue of cpiSlopeData.tif is: None
The shape of cpiSlopeData.tif is: (1, 1073, 1338)
The spatial resolution for cpiSlopeData.tif is: (0.008983152841195215, -0.008983152841195215)
The metadata for cpiSlopeData.tif is: {'scale_factor': 1.0, 'add_offset': 0.0, 'long_name': 'slope'}
1 bands found in SlopeData
Converted to dataframe!
Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927217 entries, 0 to 927216
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   latitude   927217 non-null  float64
 1   longitude  927217 non-null  float64
 2   Slope      927217 non-null  float32
dtypes: float32(1), float64(2)
memory usage: 17.7 MB
None
Adding hex info...


...

Done!
Summing within cells...
Joining to survey data...
Done!
Working with NighttimeData: 8/9...
The crs of cpiNighttimeData.tif is: EPSG:4326
The nodatavalue of cpiNighttimeData.tif is: None
The shape of cpiNighttimeData.tif is: (2, 1073, 1337)
The spatial resolution for cpiNighttimeData.tif is: (0.008983152841195215, -0.008983152841195215)
The metadata for cpiNighttimeData.tif is: {'scale_factor': 1.0, 'add_offset': 0.0, 'long_name': ('avg_rad', 'cf_cvg')}
Converted to dataframe!
Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932187 entries, 0 to 932186
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   latitude   932187 non-null  float64
 1   longitude  932187 non-null  float64
 2   avg_rad    932187 non-null  float32
 3   cf_cvg     932187 non-null  float32
dtypes: float32(2), float64(2)
memory usage: 21.3 MB
None
Adding hex info...


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Done!
Summing within cells...
Joining to survey data...
Done!
Working with GHSLData: 9/9...
The crs of cpiGHSLData.tif is: EPSG:3857
The nodatavalue of cpiGHSLData.tif is: None
The shape of cpiGHSLData.tif is: (2, 1089, 1338)
The spatial resolution for cpiGHSLData.tif is: (1000.0000000000001, -999.9999999999998)
The metadata for cpiGHSLData.tif is: {'scale_factor': 1.0, 'add_offset': 0.0, 'long_name': ('built', 'cnfd')}
Reprojection to lat/lon required: completing...
Converted to dataframe!
Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457082 entries, 0 to 1457081
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   latitude   1457082 non-null  float64
 1   longitude  1457082 non-null  float64
 2   built      1457082 non-null  float32
 3   cnfd       1457082 non-null  float32
dtypes: float32(2), float64(2)
memory usage: 33.4 MB
None
Adding hex info...


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Done!
Summing within cells...
Joining to survey data...
Done!


In [29]:
osm_nga = pd.read_csv("/Users/johnf/Downloads/raw_low_res_dssg/clean/nga_hex_osm.csv",sep="\t")
osm_nga.hex_id = osm_nga.hex_id.swifter.apply(h3.string_to_h3)
osm_nga.rename(columns={"hex_id":"hex_code"},inplace=True)
osm_nga.drop(columns=['geometry'],inplace=True)

Pandas Apply:   0%|          | 0/191094 [00:00<?, ?it/s]

In [30]:
nga_df = nga_df.merge(
    osm_nga.groupby(by=["hex_code"], as_index=False).mean(), how="left", on="hex_code"
)

In [34]:
nga_df.to_csv("/Users/johnf/Downloads/raw_low_res_dssg/clean/nga.csv", index=False)


### NOTE: used mean aggregation when lowering resolution, so for population data, the resulting estimates should be 100 times larger (100m -> 1km means 100x reduction)